<H1> ⛏ PARSING DATA FROM TWITTER
    

<H3> IMPORTS


In [ ]:
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Stream
from tweepy.streaming import StreamListener
import pandas as pd
import json
import json
import time
import sys

In [ ]:
import tweepy as tw
import pandas as pd

# API set-ups for the use of Twitter API
# Insert your keys and tokens below
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)



def get_tweets(listOfTweets, keyword, numOfTweets, data_since, data_until):
    for tweet in tw.Cursor(api.search, q=keyword+' -filter:retweets', since=data_since, until=data_until, lang='ua').items(numOfTweets):
        dict_ = {
                 'Keywords': keyword,
                 'User Name': tweet.user.name,
                 'Screen Name': tweet.user.screen_name,
                 'Tweet Created at': tweet.created_at,
                 'Tweet Text': tweet.text,
                 'Location': tweet.user.location,
                 'Likes': tweet.favorite_count,
                 'Retweets': tweet.retweet_count
                 }
        listOfTweets.append(dict_)
    return listOfTweets

list_1 = []
numberOfPosts = 100
Start_date = '2020-03-22'
End_date = '2021-06-07'
hashtags = 'новини політика'

get_tweets(list_1, hashtags, numberOfPosts, Start_date, End_date)
result_df = pd.DataFrame(list_1)   

data = result_df[['Tweet Text']]

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.to_csv('desktop/test.csv')

In [ ]:
s = [x for x in data['Tweet Text'].iloc[0:4]]

In [ ]:
s

# ➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖➖

In [ ]:
data = result_df[['Tweet Text']]
l =[data['Tweet Text'].iloc[6]]
l

In [ ]:
data.head(10)

In [ ]:
# result_df.to_csv('desktop/Wheat_Tweets.csv', index=None)
#df = pd.read_csv('desktop/Wheat_Tweets.csv')

<H1> ⛏ PARSING DATA FROM FACEBOOK

In [ ]:
pip install facebook-sdk

In [ ]:
import facebook

In [ ]:
graph = facebook.GraphAPI(access_token="EAAJOBdEMUHMBAEBsVHQ7ZAicqMpGPi0DzuYfwL2DchU4ZCaUkgC4Bas4nWvsZCjr8kgVgISr4Aqm8N5fKVRPVH3UwPNWsgv2phN6n6xRHaKEbWSAQy29JKkZB7a1xkyejKxp190Sp9J8lBSerIYt8aACmJ0gAJCBaBqgUiS1j4cBix8DjjZBP", version="2.12")

# Get the message from a post.
# post = graph.get_object(id='post_id', fields='message')
post = graph.get_object(id='https://www.facebook.com/permalink.php?story_fbid=928658477469205&id=836783153323405&__xts__[0]=68.ARAE6omhF-UgfWRI7iW_P5sU7S4vdfpwi91eXqkBjxABTuIV8YtbmPDmgvQ0WJYT0FdPp9tzTXcLWR9iPOc_Oal4PJZvL_2q5HzdUzoRZJ7tRcTVJ1OiE6OD7E0XWdtWBHomsQqBPi0c2f45-seEYDmiDvLRnIYHw4x7saF2-sIA1M0DWSkFjf0Cxe5CK9vhHgZc6j9dyntiUjtzUFdOYWrU0dvAvJ9lY6GwXjQPntbi46PO6_Lnvi9et3iUav3EaIPG8LBE-dD1o6N9AN3fxcgIYQt4AwX898UYs5Zp2iUbcKCSrTS605izIfBRROtuX_FqUgecNyEdzMzWhO0&__tn__=-R', fields='message')
print(post['message'])

# ⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️⛔️

## Another way

# Twitter


**Not working, there are some issues**

In [ ]:
class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000:
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return


    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return 

# Consumer key authentication
auth = OAuthHandler('wPIU4mm0Xt9RHpkioE8XiUjR2', 'T6zb1DKkj2WNC9F5PjpLtkX4txwsWoKLJme5LTaXSWV0LTHBQP')

# Access key authentication
auth.set_access_token('1357116781022838784-HpmRSEGOYXrRd02GZVHUltsZFeHHr2', '16VV7KEKMDokq7hIBMbBNh0DjCsNivEMAvTVA7p15gHNX')

# Set up the API with the authentication handler
api = API(auth)

# Set up words to track
def track(pattern,sourse):
    keywords_to_track = re.findall(pattern + r'\w*',sourse)
    return keywords_to_track

# Instantiate the SListener object 
listen = SListener(api)

# Instantiate the Stream object
stream = Stream(auth, listen)

# Begin collecting data
stream.filter()
#track = track(pattern,sourse)

def flatten_tweets(tweets_json):
    """ Flattens out tweet dictionaries so relevant JSON
        is in a top-level dictionary."""
    tweets_list = []
    
    # Iterate through each tweet
    for tweet in tweets_json:
        tweet_obj = json.loads(tweet)
    
        # Store the user screen name in 'user-screen_name'
        tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']
    
        # Check if this is a 140+ character tweet
        if 'extended_tweet' in tweet_obj:
            # Store the extended tweet text in 'extended_tweet-full_text'
            tweet_obj['extended_tweet-full_text'] = tweet_obj['extended_tweet']['full_text']
    
        if 'retweeted_status' in tweet_obj:
            # Store the retweet user screen name in 'retweeted_status-user-screen_name'
            tweet_obj['retweeted_status-user-screen_name'] = tweet_obj['retweeted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['retweeted_status-text'] = tweet_obj['retweeted_status']['text']
            
        tweets_list.append(tweet_obj)
    return tweets_list

# Flatten the tweets and store in `tweets`
tweets = flatten_tweets(collected_data)

# Create a DataFrame from `tweets`
ds_tweets = pd.DataFrame(tweets)